1. Single Responsibility Principle (SRP) - Принцип единственной ответственности

Задание 1

Проблема: Класс Employee занимается хранением данных о сотруднике, расчетом зарплаты с учетом налогов, генерацией отчета и сохранением отчета в файл. Это нарушение SRP, так как у класса несколько причин для изменения.

Решение: Разделить обязанности на разные классы.



In [1]:
# 1. Single Responsibility Principle (SRP)
# Задание 1

class EmployeeData:
    """
    Класс отвечает ИСКЛЮЧИТЕЛЬНО за хранение данных о сотруднике.
    """
    def __init__(self, name: str, salary: float):
        self.name = name
        self.salary = salary

class SalaryCalculator:
    """
    Класс отвечает ИСКЛЮЧИТЕЛЬНО за расчет зарплаты.
    """
    @staticmethod # Метод не зависит от состояния экземпляра, поэтому статический
    def calculate_net_salary(gross_salary: float) -> float:
        # Пример удержания налогов (можно вынести в отдельный класс/функцию, если логика сложная)
        tax_rate = 0.13 # Ставка налога (13% в РФ, в примере было 0.87 -> 13% налог)
        return gross_salary * (1 - tax_rate)

class ReportGenerator:
    """
    Класс отвечает ИСКЛЮЧИТЕЛЬНО за генерацию отчета.
    """
    @staticmethod
    def generate_employee_report(employee_data: EmployeeData, net_salary: float) -> str:
        return f"Сотрудник: {employee_data.name}, Зарплата после налогов: {net_salary:.2f}"

class FileSaver:
    """
    Класс отвечает ИСКЛЮЧИТЕЛЬНО за сохранение данных в файл.
    """
    @staticmethod
    def save_to_file(filename: str, content: str):
        try:
            with open(filename, "w", encoding="utf-8") as file:
                file.write(content)
            print(f"Отчет сохранен в файл: {filename}")
        except IOError as e:
            print(f"Ошибка при сохранении файла '{filename}': {e}")

# Демонстрация использования разделенных классов:
if __name__ == "__main__":
    print("--- 1. SRP Демонстрация ---")
    # Создаем объект с данными сотрудника
    employee_info = EmployeeData(name="Иван Иванов", salary=100000.00)

    # Рассчитываем зарплату
    net_salary_amount = SalaryCalculator.calculate_net_salary(employee_info.salary)

    # Генерируем отчет
    report_content = ReportGenerator.generate_employee_report(employee_info, net_salary_amount)
    print("Сгенерированный отчет:", report_content)

    # Сохраняем отчет в файл
    FileSaver.save_to_file("employee_report.txt", report_content)
    print("-" * 40 + "\n")

--- 1. SRP Демонстрация ---
Сгенерированный отчет: Сотрудник: Иван Иванов, Зарплата после налогов: 87000.00
Отчет сохранен в файл: employee_report.txt
----------------------------------------



2. Open/Closed Principle (OCP) - Принцип открытости/закрытости

Задание 2

Проблема: Метод process_payment в классе PaymentProcessor использует if/elif для обработки разных типов платежей. Добавление нового типа платежа (Bitcoin) потребует изменения этого метода, что нарушает OCP (класс должен быть открыт для расширения, но закрыт для изменения).

Решение: Использовать паттерн Стратегия или абстрактный класс/интерфейс для платежных систем.

In [2]:
# 2. Open/Closed Principle (OCP)
# Задание 2

from abc import ABC, abstractmethod

class IPaymentMethod(ABC):
    """
    Абстрактный интерфейс для способа оплаты.
    Классы, реализующие его, будут содержать логику конкретного способа оплаты.
    """
    @abstractmethod
    def process(self, amount: float) -> None:
        """Обрабатывает платеж на указанную сумму."""
        pass

class CreditCardPayment(IPaymentMethod):
    """Обработка платежа кредитной картой."""
    def process(self, amount: float) -> None:
        print(f"Оплата {amount:.2f} руб. через кредитную карту обработана.")

class PayPalPayment(IPaymentMethod):
    """Обработка платежа через PayPal."""
    def process(self, amount: float) -> None:
        print(f"Оплата {amount:.2f} руб. через PayPal обработана.")

# НОВЫЙ СПОСОБ ОПЛАТЫ - Bitcoin (добавляется без изменения PaymentProcessor)
class BitcoinPayment(IPaymentMethod):
    """Обработка платежа через Bitcoin."""
    def process(self, amount: float) -> None:
        # Здесь могла бы быть конвертация amount в BTC, взаимодействие с Bitcoin API и т.д.
        print(f"Оплата эквивалента {amount:.2f} руб. через Bitcoin обработана.")

class PaymentProcessorOCP:
    """
    Класс для обработки платежей, соответствующий OCP.
    Он не знает о конкретных типах платежей, а работает с интерфейсом IPaymentMethod.
    """
    def process_payment(self, payment_method: IPaymentMethod, amount: float) -> None:
        # Делегируем обработку конкретному объекту способа оплаты
        payment_method.process(amount)

# Демонстрация использования PaymentProcessorOCP:
if __name__ == "__main__":
    print("--- 2. OCP Демонстрация ---")
    processor = PaymentProcessorOCP()
    amount_to_pay = 100.50

    # Оплата кредитной картой
    credit_card_method = CreditCardPayment()
    processor.process_payment(credit_card_method, amount_to_pay)

    # Оплата через PayPal
    paypal_method = PayPalPayment()
    processor.process_payment(paypal_method, amount_to_pay + 50) # Другая сумма для примера

    # Оплата через Bitcoin (новая функциональность)
    bitcoin_method = BitcoinPayment()
    processor.process_payment(bitcoin_method, amount_to_pay + 100)
    print("-" * 40 + "\n")

--- 2. OCP Демонстрация ---
Оплата 100.50 руб. через кредитную карту обработана.
Оплата 150.50 руб. через PayPal обработана.
Оплата эквивалента 200.50 руб. через Bitcoin обработана.
----------------------------------------



Задание 3

Проблема: Класс Bicycle наследуется от Vehicle. Объект Bicycle не может корректно заменить объект Vehicle, так как вызов start_engine() для велосипеда приводит к исключению. Это нарушает LSP, который гласит, что объекты производных классов должны быть способны заменить объекты базовых классов без изменения корректности программы. У велосипеда нет двигателя, поэтому метод start_engine для него нелогичен.

Решение: Пересмотреть иерархию наследования. Возможно, не все Vehicle имеют двигатель. Можно выделить отдельный интерфейс или базовый класс для транспортных средств с двигателем.

In [3]:
# 3. Liskov Substitution Principle (LSP)
# Задание 3

from abc import ABC, abstractmethod

class VehicleLSP:
    """
    Базовый класс для транспортных средств. 
    Может иметь общие свойства, например, 'move'.
    """
    def __init__(self, name: str):
        self.name = name

    @abstractmethod
    def move(self) -> None:
        """Заставляет транспортное средство двигаться."""
        pass

class MotorizedVehicle(VehicleLSP): # Транспортное средство с мотором
    """
    Абстрактный класс для транспортных средств, имеющих двигатель.
    """
    @abstractmethod
    def start_engine(self) -> None:
        """Запускает двигатель."""
        pass

    @abstractmethod
    def stop_engine(self) -> None:
        """Останавливает двигатель."""
        pass

class Car(MotorizedVehicle):
    """Автомобиль - конкретное моторизованное транспортное средство."""
    def __init__(self, name: str, brand: str):
        super().__init__(name)
        self.brand = brand
        self._engine_running = False

    def start_engine(self) -> None:
        self._engine_running = True
        print(f"Двигатель автомобиля '{self.brand} {self.name}' запущен.")

    def stop_engine(self) -> None:
        self._engine_running = False
        print(f"Двигатель автомобиля '{self.brand} {self.name}' остановлен.")
    
    def move(self) -> None:
        if self._engine_running:
            print(f"Автомобиль '{self.brand} {self.name}' едет.")
        else:
            print(f"Автомобиль '{self.brand} {self.name}' не может ехать, двигатель не запущен.")

class BicycleLSP(VehicleLSP): # Велосипед (немоторизованный)
    """
    Велосипед - конкретное транспортное средство без двигателя.
    Он может двигаться, но у него нет метода start_engine.
    """
    def __init__(self, name: str, num_gears: int):
        super().__init__(name)
        self.num_gears = num_gears

    def move(self) -> None:
        print(f"Велосипед '{self.name}' едет (крутим педали).")

    # Метод start_engine здесь отсутствует, что логично для велосипеда.

# Функция, которая ожидает объект MotorizedVehicle
def operate_motorized_vehicle(vehicle: MotorizedVehicle):
    print(f"\n--- Управление моторизованным ТС: {vehicle.name} ---")
    vehicle.start_engine()
    vehicle.move()
    vehicle.stop_engine()

# Функция, которая ожидает любой VehicleLSP
def test_vehicle_movement(vehicle: VehicleLSP):
    print(f"\n--- Тестирование движения ТС: {vehicle.name} ---")
    vehicle.move()


# Демонстрация использования классов, соответствующих LSP:
if __name__ == "__main__":
    print("--- 3. LSP Демонстрация ---")
    my_car = Car(name="Camry", brand="Toyota")
    my_bicycle = BicycleLSP(name="Горный Велосипед", num_gears=21)

    operate_motorized_vehicle(my_car)
    # operate_motorized_vehicle(my_bicycle) # Это вызовет ошибку атрибута, так как BicycleLSP не имеет start_engine
                                        # или ошибку типизации, если используется статическая проверка типов.
                                        # Это правильно, т.к. функция ожидает моторизованное ТС.

    test_vehicle_movement(my_car)
    test_vehicle_movement(my_bicycle) # Эта функция работает для обоих, т.к. они оба VehicleLSP.
    print("-" * 40 + "\n")

--- 3. LSP Демонстрация ---

--- Управление моторизованным ТС: Camry ---
Двигатель автомобиля 'Toyota Camry' запущен.
Автомобиль 'Toyota Camry' едет.
Двигатель автомобиля 'Toyota Camry' остановлен.

--- Тестирование движения ТС: Camry ---
Автомобиль 'Toyota Camry' не может ехать, двигатель не запущен.

--- Тестирование движения ТС: Горный Велосипед ---
Велосипед 'Горный Велосипед' едет (крутим педали).
----------------------------------------



Задание 4

Проблема: Интерфейс Worker содержит методы work() и eat(). Это подходит для OfficeWorker (Офисного сотрудника), но не для Robot, который только работает и не ест. Robot вынужден реализовывать метод eat(), который для него не имеет смысла, или оставлять его пустым.

Решение: Разделить "толстый" интерфейс Worker на более мелкие и специфичные интерфейсы.

In [4]:
# 4. Interface Segregation Principle (ISP)
# Задание 4

from abc import ABC, abstractmethod

# Разделенные, более гранулярные интерфейсы
class IWorkable(ABC):
    """Интерфейс для тех, кто может работать."""
    @abstractmethod
    def work(self) -> None:
        pass

class IEatable(ABC):
    """Интерфейс для тех, кто может есть."""
    @abstractmethod
    def eat(self) -> None:
        pass

# Конкретные классы реализуют только те интерфейсы, которые им нужны
class OfficeWorkerISP(IWorkable, IEatable): # Офисный сотрудник - работает и ест
    def work(self) -> None:
        print("Офисный сотрудник усердно работает за компьютером.")

    def eat(self) -> None:
        print("Офисный сотрудник обедает в столовой.")

class RobotISP(IWorkable): # Робот - только работает
    def work(self) -> None:
        print("Робот выполняет производственные задачи на конвейере.")

    # Метод eat() здесь отсутствует, так как робот не реализует IEatable.

# Демонстрация использования разделенных интерфейсов:
if __name__ == "__main__":
    print("--- 4. ISP Демонстрация ---")
    office_staff = OfficeWorkerISP()
    factory_robot = RobotISP()

    print("\nДействия офисного сотрудника:")
    office_staff.work()
    office_staff.eat()

    print("\nДействия робота:")
    factory_robot.work()
    # factory_robot.eat() # Это вызовет ошибку атрибута, так как у робота нет метода eat().
                        # Это правильно, так как интерфейс разделен.
    
    # Функции, которые зависят от конкретных возможностей:
    def make_entity_work(worker: IWorkable):
        worker.work()

    def feed_entity(eater: IEatable):
        eater.eat()

    print("\nЗаставляем работать:")
    make_entity_work(office_staff)
    make_entity_work(factory_robot)

    print("\nКормим (только тех, кто ест):")
    feed_entity(office_staff)
    # feed_entity(factory_robot) # Ошибка, робот не IEatable
    print("-" * 40 + "\n")

--- 4. ISP Демонстрация ---

Действия офисного сотрудника:
Офисный сотрудник усердно работает за компьютером.
Офисный сотрудник обедает в столовой.

Действия робота:
Робот выполняет производственные задачи на конвейере.

Заставляем работать:
Офисный сотрудник усердно работает за компьютером.
Робот выполняет производственные задачи на конвейере.

Кормим (только тех, кто ест):
Офисный сотрудник обедает в столовой.
----------------------------------------



Задание 5

Проблема: Класс OrderProcessor напрямую зависит от конкретного класса SMSNotifier. Это создает жесткую связь. Если потребуется использовать другой способ уведомления (Email, Push), придется изменять OrderProcessor.

Решение: OrderProcessor должен зависеть от абстракции (интерфейса) уведомлений, а не от конкретной реализации. Конкретный уведомитель будет внедряться извне.

In [5]:
# 5. Dependency Inversion Principle (DIP)
# Задание 5

from abc import ABC, abstractmethod

# 1. Абстракция для уведомителя (Интерфейс)
class INotifier(ABC):
    """Абстрактный интерфейс для отправки уведомлений."""
    @abstractmethod
    def send_notification(self, message: str) -> None:
        pass

# 2. Конкретные реализации уведомителей
class SMSNotifierDIP(INotifier):
    """Конкретный уведомитель через SMS."""
    def send_notification(self, message: str) -> None:
        print(f"Отправка SMS: '{message}'")

class EmailNotifier(INotifier):
    """Конкретный уведомитель через Email."""
    def send_notification(self, message: str) -> None:
        # Здесь могла бы быть логика отправки email
        print(f"Отправка Email: '{message}'")

class PushNotifier(INotifier):
    """Конкретный уведомитель через Push-уведомления."""
    def send_notification(self, message: str) -> None:
        # Здесь могла бы быть логика отправки push-уведомления
        print(f"Отправка Push-уведомления: '{message}'")


# 3. Класс OrderProcessor, зависящий от абстракции INotifier
class OrderProcessorDIP:
    """
    Обработчик заказов, зависящий от абстракции уведомителя.
    Конкретный уведомитель внедряется через конструктор (Dependency Injection).
    """
    def __init__(self, notifier: INotifier): # Зависимость от абстракции INotifier
        self._notifier = notifier # Храним ссылку на абстрактный уведомитель

    def process_order(self, order_id: str) -> None:
        # Логика обработки заказа...
        print(f"Заказ #{order_id} успешно обработан.")
        
        # Используем внедренный уведомитель для отправки подтверждения
        confirmation_message = f"Ваш заказ #{order_id} подтвержден и принят в обработку!"
        self._notifier.send_notification(confirmation_message)

# Демонстрация использования OrderProcessorDIP с разными уведомителями:
if __name__ == "__main__":
    print("--- 5. DIP Демонстрация ---")

    # Создаем разные экземпляры уведомителей
    sms_service = SMSNotifierDIP()
    email_service = EmailNotifier()
    push_service = PushNotifier()

    # Используем OrderProcessor с SMS-уведомлениями
    order_processor_sms = OrderProcessorDIP(notifier=sms_service)
    order_processor_sms.process_order("ЗАКАЗ-001")

    print("---")
    # Используем OrderProcessor с Email-уведомлениями
    # Мы не меняем код OrderProcessorDIP, а просто передаем другой уведомитель
    order_processor_email = OrderProcessorDIP(notifier=email_service)
    order_processor_email.process_order("ЗАКАЗ-002")

    print("---")
    # Используем OrderProcessor с Push-уведомлениями
    order_processor_push = OrderProcessorDIP(notifier=push_service)
    order_processor_push.process_order("ЗАКАЗ-003")
    print("-" * 40 + "\n")

--- 5. DIP Демонстрация ---
Заказ #ЗАКАЗ-001 успешно обработан.
Отправка SMS: 'Ваш заказ #ЗАКАЗ-001 подтвержден и принят в обработку!'
---
Заказ #ЗАКАЗ-002 успешно обработан.
Отправка Email: 'Ваш заказ #ЗАКАЗ-002 подтвержден и принят в обработку!'
---
Заказ #ЗАКАЗ-003 успешно обработан.
Отправка Push-уведомления: 'Ваш заказ #ЗАКАЗ-003 подтвержден и принят в обработку!'
----------------------------------------

